# FIFA World Cup Match Predictions Using Neural Networks

### CS109A

**Nathan Goldberg, Erik Johnsson, James Weichert**

**August, 2018**

<hr style="height:2pt">

In [1]:
import numpy as np
import pandas as pd
import keras
import statsmodels as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy import stats

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.layers import BatchNormalization

## Data: International Men's Soccer Matches (2006-2018)

### FIFA

In [80]:
#Import the match data with the FIFA rankings
fifa_df = pd.read_csv("nb4a_fifa_updated_wins.csv", ",", index_col=0)

In [4]:
fifa_df.head()

,FT H,FT G,AET H,AET G,Type,HomeAdv,Win,PrevDiffHome1,PrevDiffAway1,PrevDiffHome2,...,dist_home,dist_away,TravelHome,TravelAway,TZDeltaHome,TZDeltaAway,team_a_ranking,team_a_points,team_b_ranking,team_b_points
0,2,1,NaN,NaN,FSS,1,1.0,1,-1,1,...,109.811860,9305.286395,17.045896,72.694317,0.0,0.000000,3.0,0.0,4.0,0.0
1,0,0,NaN,NaN,FSS,1,0.0,0,2,-1,...,0.000000,7737.037490,0.000000,1289.506248,0.0,-1.166667,40.0,0.0,29.0,0.0
2,0,1,NaN,NaN,FSS,1,-1.0,-1,-1,-1,...,0.000000,4613.786111,0.000000,0.000000,0.0,0.000000,123.0,0.0,61.0,0.0
3,1,1,NaN,NaN,FSS,1,0.0,0,0,-3,...,0.000000,13064.115395,49.689910,490.153628,0.0,0.000000,16.0,0.0,64.0,0.0
4,3,0,NaN,NaN,FSS,1,1.0,0,1,7,...,80.068621,8452.618598,14.235933,296.477215,0.0,0.000000,19.0,0.0,27.0,0.0


In [5]:
#Separate into a predictor DataFrame and response variable column
y_train_fifa = fifa_df['Win']

#Drop the response variable from the predictor DataFrame. Also drop 'Type', as the baseline models will not take
#the type of game into account
X_train_fifa = fifa_df.drop(['Win', 'FT H', 'FT G', 'AET H', 'AET G'], axis=1)

In [6]:
X_train_fifa = pd.concat([X_train_fifa, pd.get_dummies(X_train_fifa['Type'])], axis=1).drop(['Type'], axis=1)

In [7]:
X_train_fifa

,HomeAdv,PrevDiffHome1,PrevDiffAway1,PrevDiffHome2,PrevDiffAway2,PrevDiffHome3,PrevDiffAway3,PrevDiffHome4,PrevDiffAway4,PrevDiffHome5,...,team_a_ranking,team_a_points,team_b_ranking,team_b_points,Cont,ContQ,FSS,KFC,WM,WM-Q
0,1,1,-1,1,1,-2,1,0,1,2,...,3.0,0.00,4.0,0.00,0,0,1,0,0,0
1,1,0,2,-1,0,-5,1,-1,1,-1,...,40.0,0.00,29.0,0.00,0,0,1,0,0,0
2,1,-1,-1,-1,-1,3,-2,-2,-2,0,...,123.0,0.00,61.0,0.00,0,0,1,0,0,0
3,1,0,0,-3,1,0,1,0,3,0,...,16.0,0.00,64.0,0.00,0,0,1,0,0,0
4,1,0,1,7,0,3,0,-3,0,0,...,19.0,0.00,27.0,0.00,0,0,1,0,0,0
5,1,3,1,-1,0,0,2,-3,1,2,...,21.0,0.00,22.0,0.00,0,0,1,0,0,0
6,0,-1,-3,0,0,-1,-1,-3,2,-1,...,86.0,0.00,65.0,0.00,0,0,1,0,0,0
7,0,-2,1,2,0,4,0,-1,0,1,...,57.0,0.00,14.0,0.00,0,0,1,0,0,0
8,0,0,4,3,0,0,1,2,0,1,...,13.0,0.00,45.0,0.00,0,0,1,0,0,0
9,1,0,0,1,1,0,3,4,2,6,...,5.0,0.00,17.0,0.00,0,0,1,0,0,0


In [8]:
y_train_fifa = pd.DataFrame(y_train_fifa)

#Make a copy of the current state of y_train for use in Logistic Regression (which requires a different kind of 
#categorical encoding than the neural network)
y_train_fifa_cat = y_train_fifa.copy()

W = np.array(np.zeros(len(y_train_fifa)))
D = np.array(np.zeros(len(y_train_fifa)))
L = np.array(np.zeros(len(y_train_fifa)))

#Convert the {-1, 0, 1} values in the response variable to one-hot encoded columns for win (home win), draw, and loss
#(away win)
for y in range(len(y_train_fifa)):
    if y_train_fifa.iloc[y].Win == 1:
        W[y] = 1
    elif y_train_fifa.iloc[y].Win == 0:
        D[y] = 1
    elif y_train_fifa.iloc[y].Win == -1:
        L[y] = 1

#Add the one-hot encoded columns to the response variable DataFrame and delete the original 'Win' column
y_train_fifa['W'] = W
y_train_fifa['D'] = D
y_train_fifa['L'] = L

y_train_fifa = y_train_fifa.drop(['Win'], axis=1)

In [9]:
y_train_fifa

,W,D,L
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0
6,0.0,1.0,0.0
7,0.0,0.0,1.0
8,0.0,1.0,0.0
9,1.0,0.0,0.0


---
### WIGGO
---

In [81]:
#Import the match data with the FIFA rankings
wiggo_df = pd.read_csv("nb4b_wiggo_updated_wins.csv", ",", index_col=0)

In [11]:
wiggo_df.head()

,FT H,FT G,AET H,AET G,Type,HomeAdv,Win,PrevDiffHome1,PrevDiffAway1,PrevDiffHome2,...,dist_home,dist_away,TravelHome,TravelAway,TZDeltaHome,TZDeltaAway,team_a_ranking,team_a_points,team_b_ranking,team_b_points
0,2,1,NaN,NaN,FSS,1,1.0,1,-1,1,...,109.811860,9305.286395,17.045896,72.694317,0.0,0.000000,3.0,1828.100435,11.0,1764.714989
1,0,0,NaN,NaN,FSS,1,0.0,0,2,-1,...,0.000000,7737.037490,0.000000,1289.506248,0.0,-1.166667,39.0,1659.123301,38.0,1661.105272
2,0,1,NaN,NaN,FSS,1,-1.0,-1,-1,-1,...,0.000000,4613.786111,0.000000,0.000000,0.0,0.000000,168.0,1267.787070,87.0,1504.624820
3,1,1,NaN,NaN,FSS,1,0.0,0,0,-3,...,0.000000,13064.115395,49.689910,490.153628,0.0,0.000000,13.0,1754.982681,49.0,1619.366237
4,3,0,NaN,NaN,FSS,1,1.0,0,1,7,...,80.068621,8452.618598,14.235933,296.477215,0.0,0.000000,10.0,1767.390337,36.0,1670.498015


In [12]:
#Separate into a predictor DataFrame and response variable column
y_train_wiggo = wiggo_df['Win']

#Drop the response variable from the predictor DataFrame. Also drop 'Type', as the baseline models will not take
#the type of game into account
X_train_wiggo = wiggo_df.drop(['Win', 'FT H', 'FT G', 'AET H', 'AET G'], axis=1)

In [13]:
X_train_wiggo = pd.concat([X_train_wiggo, pd.get_dummies(X_train_wiggo['Type'])], axis=1).drop(['Type'], axis=1)

In [14]:
y_train_wiggo = pd.DataFrame(y_train_wiggo)

#Make a copy of the current state of y_train for use in Logistic Regression (which requires a different kind of 
#categorical encoding than the neural network)
y_train_wiggo_cat = y_train_wiggo.copy()

W = np.array(np.zeros(len(y_train_wiggo)))
D = np.array(np.zeros(len(y_train_wiggo)))
L = np.array(np.zeros(len(y_train_wiggo)))

#Convert the {0, 0.5, 1} values in the response variable to one-hot encoded columns for win (home win), draw, and loss
#(away win)
for y in range(len(y_train_wiggo)):
    if y_train_wiggo.iloc[y].Win == 1:
        W[y] = 1
    elif y_train_wiggo.iloc[y].Win == 0:
        D[y] = 1
    elif y_train_wiggo.iloc[y].Win == -1:
        L[y] = 1

#Add the one-hot encoded columns to the response variable DataFrame and delete the original 'Win' column
y_train_wiggo['W'] = W
y_train_wiggo['D'] = D
y_train_wiggo['L'] = L

y_train_wiggo = y_train_wiggo.drop(['Win'], axis=1)

In [15]:
y_train_wiggo

,W,D,L
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0
6,0.0,1.0,0.0
7,0.0,0.0,1.0
8,0.0,1.0,0.0
9,1.0,0.0,0.0


---

### World Cup Test Data (FIFA)

---

In [82]:
#Read in the .csv containing the World Cup fixtures and the corresponding statistics (statistics are accurate
#up to the start of the match)

fixtures_fifa_df = pd.read_csv("nb4c_wc_fixtures.csv", ",", index_col=0)

fixtures_fifa_df.head()

,Type,HomeAdv,Win,PrevDiffHome1,PrevDiffAway1,PrevDiffHome2,PrevDiffAway2,PrevDiffHome3,PrevDiffAway3,PrevDiffHome4,...,dist_home,dist_away,TravelHome,TravelAway,TZDeltaHome,TZDeltaAway,team_a_ranking,team_a_points,team_b_ranking,team_b_points
14248,WM,0,1.0,0,-1,-1,-3,-2,-1,-3,...,0.000000,3534.539231,0.000000,347.574603,0,0,70,456.53,67,465.28
14249,WM,0,0.0,-3,3,0,1,0,2,-1,...,3750.413399,14770.693705,405.969902,1846.336713,0,0,45,649.43,14,1018.41
14250,WM,0,0.0,2,1,1,-1,0,1,2,...,4224.038712,3092.755264,53.101968,90.878998,0,0,41,685.86,211,0.00
14251,WM,0,0.5,3,1,0,0,0,5,-3,...,4077.831369,3573.971457,509.728921,28.632563,0,0,4,1273.69,10,1125.50
14252,WM,0,1.0,0,1,2,4,2,0,2,...,3214.759841,13767.992688,464.575231,322.033967,0,0,7,1198.13,36,718.33


In [17]:
#Separate X and y
y_test_fifa = fixtures_fifa_df['Win']
X_test_fifa = fixtures_fifa_df.drop(['Win'], axis=1)

In [83]:
X_fifa = pd.read_csv("nb4a_fifa_updated_wins.csv", ",", index_col=0).drop(['Win'], axis=1)

In [19]:
X_fifa = X_fifa.drop(['FT H', 'FT G', 'AET H', 'AET G'], axis=1)

In [20]:
all_matches_fifa = pd.concat([X_fifa, X_test_fifa], axis=0)

In [21]:
all_matches_fifa_dummies = pd.concat([all_matches_fifa, pd.get_dummies(all_matches_fifa['Type'])], axis=1)

In [22]:
X_test_fifa = all_matches_fifa_dummies.iloc[len(all_matches_fifa_dummies)-64:].drop(['Type'], axis=1)

---

### World Cup Test Data (WIGGO)

---

In [84]:
#Read in the .csv containing the World Cup fixtures and the corresponding statistics (statistics are accurate
#up to the start of the match)

fixtures_wiggo_df = pd.read_csv("nb4d_wc_wiggo_df.csv", ",", index_col=0)

fixtures_wiggo_df.head()

,Type,HomeAdv,Win,PrevDiffHome1,PrevDiffAway1,PrevDiffHome2,PrevDiffAway2,PrevDiffHome3,PrevDiffAway3,PrevDiffHome4,...,dist_home,dist_away,TravelHome,TravelAway,TZDeltaHome,TZDeltaAway,team_a_ranking,team_a_points,team_b_ranking,team_b_points
14248,WM,1,1.0,0,-1,-1,-3,-2,-1,-3,...,0.000000,3534.539231,0.000000,347.574603,0.000000,0.166667,45,1639.144591,56,1611.629202
14249,WM,0,0.0,-3,3,0,1,0,2,-1,...,3750.413399,14770.693705,405.969902,1846.336713,0.333333,1.000000,52,1622.222198,13,1778.383238
14250,WM,0,0.0,2,1,1,-1,0,1,2,...,4224.038712,3092.755264,53.101968,90.878998,0.000000,0.000000,39,1665.448222,19,1730.381002
14251,WM,0,0.5,3,1,0,0,0,5,-3,...,4077.831369,3573.971457,509.728921,28.632563,0.250000,0.000000,5,1841.546136,3,1886.162292
14252,WM,0,1.0,0,1,2,4,2,0,2,...,3214.759841,13767.992688,464.575231,322.033967,0.142857,0.142857,6,1824.543984,30,1680.660285


In [24]:
#Separate X and y
y_test_wiggo = fixtures_wiggo_df['Win']
X_test_wiggo = fixtures_wiggo_df.drop(['Win'], axis=1)

In [85]:
X_wiggo = pd.read_csv("nb4b_wiggo_updated_wins.csv", ",", index_col=0).drop(['Win'], axis=1)

In [26]:
X_wiggo = X_wiggo.drop(['FT H', 'FT G', 'AET H', 'AET G'], axis=1)

In [27]:
all_matches_wiggo = pd.concat([X_wiggo, X_test_wiggo], axis=0)

In [28]:
all_matches_wiggo_dummies = pd.concat([all_matches_wiggo, pd.get_dummies(all_matches_wiggo['Type'])], axis=1)

In [29]:
X_test_wiggo = all_matches_wiggo_dummies.iloc[len(all_matches_wiggo_dummies)-64:].drop(['Type'], axis=1)

---
## Neural Network
---

In [50]:
np.random.seed(201)

In [51]:
#Create keras neural network model
model = Sequential()

In [52]:
#Add layers to the network
model.add(Dense(units=46, activation='relu', kernel_initializer='random_uniform', input_dim=37))
model.add(Dense(units=29, activation='relu', kernel_initializer='random_uniform'))
model.add(Dense(units=17, activation='relu', kernel_initializer='random_uniform'))
model.add(Dense(units=8, activation='relu', kernel_initializer='random_uniform'))
#model.add(Dense(units=20, activation='relu', kernel_initializer='random_uniform'))
#model.add(Dense(units=10, activation='relu', kernel_initializer='random_uniform'))
#Since this is a classification problem (win, draw, loss), a softmax will be used as the final activation
model.add(Dense(units=3, activation='softmax', kernel_initializer='random_uniform'))

In [53]:
#Compile the model using categorical cross-entropy as a loss function with a categorical_accuracy metric. Use the 
#'adam' optimizer for faster convergence on a minimum
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [54]:
#Train the model
model.fit(X_train_wiggo, y_train_wiggo, epochs=50, batch_size=32,verbose=1, validation_split = .2)
print("Done")

Train on 8724 samples, validate on 2181 samples
Epoch 1/100
8724/8724 [==============================] - 3s 323us/step - loss: 1.0436 - categorical_accuracy: 0.4773 - val_loss: 0.9647 - val_categorical_accuracy: 0.5456
Epoch 2/100
8724/8724 [==============================] - 1s 86us/step - loss: 0.9417 - categorical_accuracy: 0.5582 - val_loss: 0.9079 - val_categorical_accuracy: 0.5846
Epoch 3/100
8724/8724 [==============================] - 1s 88us/step - loss: 0.9161 - categorical_accuracy: 0.5679 - val_loss: 0.8968 - val_categorical_accuracy: 0.5887
Epoch 4/100
8724/8724 [==============================] - 1s 89us/step - loss: 0.9114 - categorical_accuracy: 0.5679 - val_loss: 0.8973 - val_categorical_accuracy: 0.5906
Epoch 5/100
8724/8724 [==============================] - 1s 80us/step - loss: 0.9172 - categorical_accuracy: 0.5648 - val_loss: 0.8943 - val_categorical_accuracy: 0.5896
Epoch 6/100
8724/8724 [==============================] - 1s 83us/step - loss: 0.9093 - categorical_ac

Epoch 49/100
8724/8724 [==============================] - 1s 83us/step - loss: 0.8943 - categorical_accuracy: 0.5784 - val_loss: 0.8937 - val_categorical_accuracy: 0.5878
Epoch 50/100
8724/8724 [==============================] - 1s 80us/step - loss: 0.8933 - categorical_accuracy: 0.5822 - val_loss: 0.9054 - val_categorical_accuracy: 0.5796
Epoch 51/100
8724/8724 [==============================] - 1s 89us/step - loss: 0.8945 - categorical_accuracy: 0.5808 - val_loss: 0.8885 - val_categorical_accuracy: 0.5860
Epoch 52/100
8724/8724 [==============================] - 1s 103us/step - loss: 0.8938 - categorical_accuracy: 0.5785 - val_loss: 0.8962 - val_categorical_accuracy: 0.5906
Epoch 53/100
8724/8724 [==============================] - 1s 104us/step - loss: 0.8928 - categorical_accuracy: 0.5808 - val_loss: 0.8907 - val_categorical_accuracy: 0.5901
Epoch 54/100
8724/8724 [==============================] - 1s 94us/step - loss: 0.8950 - categorical_accuracy: 0.5781 - val_loss: 0.8932 - val_c

Epoch 97/100
8724/8724 [==============================] - 1s 89us/step - loss: 0.8865 - categorical_accuracy: 0.5810 - val_loss: 0.9062 - val_categorical_accuracy: 0.5823
Epoch 98/100
8724/8724 [==============================] - 1s 79us/step - loss: 0.8828 - categorical_accuracy: 0.5872 - val_loss: 0.9068 - val_categorical_accuracy: 0.5951
Epoch 99/100
8724/8724 [==============================] - 1s 80us/step - loss: 0.8836 - categorical_accuracy: 0.5825 - val_loss: 0.9165 - val_categorical_accuracy: 0.5800
Epoch 100/100
8724/8724 [==============================] - 1s 78us/step - loss: 0.8829 - categorical_accuracy: 0.5833 - val_loss: 0.9099 - val_categorical_accuracy: 0.5860
Done


In [55]:
#Generate neural network predictions for the test set
predictions = pd.DataFrame(model.predict(X_test_wiggo, batch_size=5), columns=['Win', 'Draw', 'Loss'])

In [56]:
#Sanity check: make sure the max values for each categorical column are close to 1, which idicates a high confidence
#that at least one game will end in a win, draw, or loss (respectively)
predictions.describe()

,Win,Draw,Loss
count,6.400000e+01,6.400000e+01,6.400000e+01
mean,4.083647e-01,2.422973e-01,3.493381e-01
std,2.174072e-01,7.939528e-02,2.044795e-01
min,9.920524e-13,6.110897e-07,1.928693e-11
25%,2.669481e-01,2.175186e-01,1.730078e-01
50%,3.493366e-01,2.645542e-01,3.281861e-01
75%,5.749508e-01,2.947084e-01,4.720138e-01
max,9.999992e-01,3.234785e-01,9.999994e-01


In [57]:
#Convert the on-hot encoding back into a single column
results = [0]*len(predictions)

for i in range(len(predictions)):
    if(predictions.iloc[i].Win > predictions.iloc[i].Draw and predictions.iloc[i].Win > predictions.iloc[i].Loss):
        results[i] = 1
    elif(predictions.iloc[i].Draw >= predictions.iloc[i].Win and predictions.iloc[i].Draw >= predictions.iloc[i].Loss):
        results[i] = 0.5
    elif(predictions.iloc[i].Loss > predictions.iloc[i].Win and predictions.iloc[i].Loss > predictions.iloc[i].Draw):
        results[i] = 0
        

In [60]:
#Count how many times the prediction is correct and print the percentage of correct predictions
nn_correct_predictions = 0

for i in range(len(y_test_wiggo)):
    if (y_test_wiggo.iloc[i] == results[i]):
        nn_correct_predictions += 1

print("Neural Network Prediction Accuracy Using WIGGO (2018 World Cup): {0}%".format(100*nn_correct_predictions/len(y_test_wiggo)))

Neural Network Prediction Accuracy (2018 World Cup): 62.5%


---

## Generate Multiple Prediction Accuracies (Average 10 Neural Network Accuracies)

---

In [67]:
def run_nn(X_train, y_train, e=50):
    #Create keras neural network model
    model = Sequential()
    
    #Add layers to the network
    model.add(Dense(units=29, activation='relu', kernel_initializer='random_uniform', input_dim=37))
    #model.add(Dense(units=39, activation='relu', kernel_initializer='random_uniform', input_dim=31))
    #model.add(Dense(units=21, activation='relu', kernel_initializer='random_uniform'))
    #Since this is a classification problem (win, draw, loss), a softmax will be used as the final activation
    model.add(Dense(units=3, activation='softmax', kernel_initializer='random_uniform'))
    
    #Compile the model using categorical cross-entropy as a loss function with a categorical_accuracy metric. Use the 
    #'adam' optimizer for faster convergence on a minimum
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])
    
    #Train the model
    model.fit(X_train, y_train, epochs=e, batch_size=32,verbose=1, validation_split = .2)
    print("Done")
    
    return model

In [68]:
def test_nn(trained_model, X_test, y_test, data_type):
    #Generate neural network predictions for the test set
    predictions = pd.DataFrame(trained_model.predict(X_test, batch_size=5), columns=['Win', 'Draw', 'Loss'])
    
    #Sanity check: make sure the max values for each categorical column are close to 1, which idicates a high confidence
    #that at least one game will end in a win, draw, or loss (respectively)
    #print(predictions.describe())
    
    #Convert the on-hot encoding back into a single column
    results = [0]*len(predictions)

    for i in range(len(predictions)):
        if(predictions.iloc[i].Win > predictions.iloc[i].Draw and predictions.iloc[i].Win > predictions.iloc[i].Loss):
            results[i] = 1
        elif(predictions.iloc[i].Draw >= predictions.iloc[i].Win and predictions.iloc[i].Draw >= predictions.iloc[i].Loss):
            results[i] = 0.5
        elif(predictions.iloc[i].Loss > predictions.iloc[i].Win and predictions.iloc[i].Loss > predictions.iloc[i].Draw):
            results[i] = 0
            
    #Count how many times the prediction is correct and print the percentage of correct predictions
    nn_correct_predictions = 0

    for i in range(len(y_test)):
        if (y_test.iloc[i] == results[i]):
            nn_correct_predictions += 1

    print("Neural Network Prediction Accuracy ({0}): {1}%".format(data_type, 100*nn_correct_predictions/len(y_test)))
    
    return 100*nn_correct_predictions/len(y_test)

In [69]:
sample_wiggo = [np.nan]*10

for sample in range(len(sample_wiggo)):
    model = run_nn(X_train_wiggo, y_train_wiggo, e=50)
    
    sample_wiggo[sample] = test_nn(model, X_test_wiggo, y_test_wiggo, "WIGGO")

Train on 8724 samples, validate on 2181 samples
Epoch 1/50
8724/8724 [==============================] - 3s 349us/step - loss: 1.4813 - categorical_accuracy: 0.4739 - val_loss: 0.9876 - val_categorical_accuracy: 0.5608
Epoch 2/50
8724/8724 [==============================] - 1s 92us/step - loss: 1.0955 - categorical_accuracy: 0.5282 - val_loss: 0.9528 - val_categorical_accuracy: 0.5896
Epoch 3/50
8724/8724 [==============================] - 1s 89us/step - loss: 0.9788 - categorical_accuracy: 0.5501 - val_loss: 0.9270 - val_categorical_accuracy: 0.5828
Epoch 4/50
8724/8724 [==============================] - 1s 102us/step - loss: 0.9768 - categorical_accuracy: 0.5531 - val_loss: 0.9204 - val_categorical_accuracy: 0.5681
Epoch 5/50
8724/8724 [==============================] - 1s 91us/step - loss: 0.9571 - categorical_accuracy: 0.5565 - val_loss: 0.9261 - val_categorical_accuracy: 0.5695
Epoch 6/50
8724/8724 [==============================] - 1s 101us/step - loss: 0.9409 - categorical_accura

Epoch 49/50
8724/8724 [==============================] - 1s 169us/step - loss: 0.9094 - categorical_accuracy: 0.5708 - val_loss: 0.8968 - val_categorical_accuracy: 0.5951
Epoch 50/50
8724/8724 [==============================] - 1s 118us/step - loss: 0.9084 - categorical_accuracy: 0.5665 - val_loss: 0.9092 - val_categorical_accuracy: 0.5878
Done
Neural Network Prediction Accuracy (WIGGO): 60.9375%
Train on 8724 samples, validate on 2181 samples
Epoch 1/50
8724/8724 [==============================] - 3s 382us/step - loss: 2.1011 - categorical_accuracy: 0.4660 - val_loss: 0.9825 - val_categorical_accuracy: 0.5374
Epoch 2/50
8724/8724 [==============================] - 1s 95us/step - loss: 1.0640 - categorical_accuracy: 0.5269 - val_loss: 1.0366 - val_categorical_accuracy: 0.5786
Epoch 3/50
8724/8724 [==============================] - 1s 88us/step - loss: 1.0269 - categorical_accuracy: 0.5389 - val_loss: 0.9179 - val_categorical_accuracy: 0.5814
Epoch 4/50
8724/8724 [======================

8724/8724 [==============================] - 1s 100us/step - loss: 0.9075 - categorical_accuracy: 0.5773 - val_loss: 0.8992 - val_categorical_accuracy: 0.5864
Epoch 47/50
8724/8724 [==============================] - 1s 170us/step - loss: 0.9078 - categorical_accuracy: 0.5757 - val_loss: 0.8937 - val_categorical_accuracy: 0.5864
Epoch 48/50
8724/8724 [==============================] - 1s 154us/step - loss: 0.9130 - categorical_accuracy: 0.5719 - val_loss: 0.9056 - val_categorical_accuracy: 0.5796
Epoch 49/50
8724/8724 [==============================] - 1s 171us/step - loss: 0.9085 - categorical_accuracy: 0.5768 - val_loss: 0.9098 - val_categorical_accuracy: 0.5653
Epoch 50/50
8724/8724 [==============================] - 1s 119us/step - loss: 0.9110 - categorical_accuracy: 0.5739 - val_loss: 0.8959 - val_categorical_accuracy: 0.5919
Done
Neural Network Prediction Accuracy (WIGGO): 59.375%
Train on 8724 samples, validate on 2181 samples
Epoch 1/50
8724/8724 [==============================

8724/8724 [==============================] - 1s 84us/step - loss: 0.9103 - categorical_accuracy: 0.5721 - val_loss: 0.9116 - val_categorical_accuracy: 0.5832
Epoch 44/50
8724/8724 [==============================] - 1s 82us/step - loss: 0.9111 - categorical_accuracy: 0.5684 - val_loss: 0.8928 - val_categorical_accuracy: 0.5823
Epoch 45/50
8724/8724 [==============================] - 1s 83us/step - loss: 0.9107 - categorical_accuracy: 0.5734 - val_loss: 0.8971 - val_categorical_accuracy: 0.5901
Epoch 46/50
8724/8724 [==============================] - 1s 81us/step - loss: 0.9080 - categorical_accuracy: 0.5727 - val_loss: 0.8941 - val_categorical_accuracy: 0.5910
Epoch 47/50
8724/8724 [==============================] - 1s 121us/step - loss: 0.9115 - categorical_accuracy: 0.5723 - val_loss: 0.9037 - val_categorical_accuracy: 0.5873
Epoch 48/50
8724/8724 [==============================] - 1s 109us/step - loss: 0.9103 - categorical_accuracy: 0.5726 - val_loss: 0.8963 - val_categorical_accurac

8724/8724 [==============================] - 1s 99us/step - loss: 0.9078 - categorical_accuracy: 0.5745 - val_loss: 0.8976 - val_categorical_accuracy: 0.5722
Epoch 41/50
8724/8724 [==============================] - 1s 91us/step - loss: 0.9094 - categorical_accuracy: 0.5738 - val_loss: 0.8903 - val_categorical_accuracy: 0.5896
Epoch 42/50
8724/8724 [==============================] - 1s 91us/step - loss: 0.9095 - categorical_accuracy: 0.5745 - val_loss: 0.8905 - val_categorical_accuracy: 0.5869
Epoch 43/50
8724/8724 [==============================] - 1s 89us/step - loss: 0.9107 - categorical_accuracy: 0.5692 - val_loss: 0.8946 - val_categorical_accuracy: 0.5878
Epoch 44/50
8724/8724 [==============================] - 1s 90us/step - loss: 0.9074 - categorical_accuracy: 0.5732 - val_loss: 0.8962 - val_categorical_accuracy: 0.5942
Epoch 45/50
8724/8724 [==============================] - 1s 90us/step - loss: 0.9088 - categorical_accuracy: 0.5716 - val_loss: 0.9250 - val_categorical_accuracy:

...

8724/8724 [==============================] - 1s 98us/step - loss: 0.9122 - categorical_accuracy: 0.5702 - val_loss: 0.9092 - val_categorical_accuracy: 0.5896
Epoch 35/50
8724/8724 [==============================] - 1s 88us/step - loss: 0.9138 - categorical_accuracy: 0.5736 - val_loss: 0.8954 - val_categorical_accuracy: 0.5928
Epoch 36/50
8724/8724 [==============================] - 1s 88us/step - loss: 0.9081 - categorical_accuracy: 0.5719 - val_loss: 0.8957 - val_categorical_accuracy: 0.5878
Epoch 37/50
8724/8724 [==============================] - 1s 90us/step - loss: 0.9104 - categorical_accuracy: 0.5704 - val_loss: 0.8941 - val_categorical_accuracy: 0.5906
Epoch 38/50
8724/8724 [==============================] - 1s 85us/step - loss: 0.9159 - categorical_accuracy: 0.5669 - val_loss: 0.9144 - val_categorical_accuracy: 0.5906
Epoch 39/50
8724/8724 [==============================] - 1s 93us/step - loss: 0.9137 - categorical_accuracy: 0.5687 - val_loss: 0.8935 - val_categorical_accuracy:

8724/8724 [==============================] - 1s 87us/step - loss: 0.9066 - categorical_accuracy: 0.5739 - val_loss: 0.9029 - val_categorical_accuracy: 0.5910
Epoch 32/50
8724/8724 [==============================] - 1s 92us/step - loss: 0.9085 - categorical_accuracy: 0.5750 - val_loss: 0.8945 - val_categorical_accuracy: 0.5896
Epoch 33/50
8724/8724 [==============================] - 1s 87us/step - loss: 0.9103 - categorical_accuracy: 0.5762 - val_loss: 0.8913 - val_categorical_accuracy: 0.5942
Epoch 34/50
8724/8724 [==============================] - 1s 82us/step - loss: 0.9100 - categorical_accuracy: 0.5702 - val_loss: 0.9001 - val_categorical_accuracy: 0.5915
Epoch 35/50
8724/8724 [==============================] - 1s 85us/step - loss: 0.9145 - categorical_accuracy: 0.5719 - val_loss: 0.8942 - val_categorical_accuracy: 0.5883
Epoch 36/50
8724/8724 [==============================] - 1s 84us/step - loss: 0.9062 - categorical_accuracy: 0.5747 - val_loss: 0.9056 - val_categorical_accuracy:

8724/8724 [==============================] - 1s 86us/step - loss: 0.9096 - categorical_accuracy: 0.5712 - val_loss: 0.8927 - val_categorical_accuracy: 0.5892
Epoch 29/50
8724/8724 [==============================] - 1s 87us/step - loss: 0.9115 - categorical_accuracy: 0.5739 - val_loss: 0.9215 - val_categorical_accuracy: 0.5805
Epoch 30/50
8724/8724 [==============================] - 1s 122us/step - loss: 0.9108 - categorical_accuracy: 0.5747 - val_loss: 0.9034 - val_categorical_accuracy: 0.5910
Epoch 31/50
8724/8724 [==============================] - 1s 107us/step - loss: 0.9113 - categorical_accuracy: 0.5718 - val_loss: 0.8946 - val_categorical_accuracy: 0.5878
Epoch 32/50
8724/8724 [==============================] - 1s 96us/step - loss: 0.9137 - categorical_accuracy: 0.5676 - val_loss: 0.8966 - val_categorical_accuracy: 0.5828
Epoch 33/50
8724/8724 [==============================] - 1s 113us/step - loss: 0.9164 - categorical_accuracy: 0.5692 - val_loss: 0.8979 - val_categorical_accura

8724/8724 [==============================] - 1s 133us/step - loss: 0.9092 - categorical_accuracy: 0.5770 - val_loss: 0.8943 - val_categorical_accuracy: 0.5846
Epoch 26/50
8724/8724 [==============================] - 1s 108us/step - loss: 0.9094 - categorical_accuracy: 0.5768 - val_loss: 0.8921 - val_categorical_accuracy: 0.5896
Epoch 27/50
8724/8724 [==============================] - 1s 100us/step - loss: 0.9115 - categorical_accuracy: 0.5775 - val_loss: 0.8952 - val_categorical_accuracy: 0.5942
Epoch 28/50
8724/8724 [==============================] - 1s 97us/step - loss: 0.9073 - categorical_accuracy: 0.5760 - val_loss: 0.8978 - val_categorical_accuracy: 0.5851
Epoch 29/50
8724/8724 [==============================] - 1s 104us/step - loss: 0.9091 - categorical_accuracy: 0.5726 - val_loss: 0.8975 - val_categorical_accuracy: 0.5864
Epoch 30/50
8724/8724 [==============================] - 1s 104us/step - loss: 0.9122 - categorical_accuracy: 0.5674 - val_loss: 0.8957 - val_categorical_accu

8724/8724 [==============================] - 1s 101us/step - loss: 0.9101 - categorical_accuracy: 0.5738 - val_loss: 0.8895 - val_categorical_accuracy: 0.5919
Epoch 23/50
8724/8724 [==============================] - 1s 87us/step - loss: 0.9114 - categorical_accuracy: 0.5696 - val_loss: 0.8987 - val_categorical_accuracy: 0.5892
Epoch 24/50
8724/8724 [==============================] - 1s 156us/step - loss: 0.9101 - categorical_accuracy: 0.5716 - val_loss: 0.8916 - val_categorical_accuracy: 0.5910
Epoch 25/50
8724/8724 [==============================] - 1s 115us/step - loss: 0.9132 - categorical_accuracy: 0.5708 - val_loss: 0.8909 - val_categorical_accuracy: 0.5938
Epoch 26/50
8724/8724 [==============================] - 1s 100us/step - loss: 0.9085 - categorical_accuracy: 0.5757 - val_loss: 0.9008 - val_categorical_accuracy: 0.5823
Epoch 27/50
8724/8724 [==============================] - 1s 117us/step - loss: 0.9076 - categorical_accuracy: 0.5762 - val_loss: 0.9029 - val_categorical_accu

In [70]:
sample_wiggo

[60.9375, 59.375, 62.5, 57.8125, 62.5, 56.25, 54.6875, 53.125, 53.125, 50.0]

In [76]:
print("WIGGO Average Test Accuracy (10 Identical Neural Networks): {0}%".format(np.mean(sample_wiggo)))

WIGGO Average Test Accuracy (10 Identical Neural Networks): 57.03125%


In [72]:
sample_fifa = [np.nan]*10

for sample in range(len(sample_fifa)):
    model = run_nn(X_train_fifa, y_train_fifa, e=50)
    
    sample_fifa[sample] = test_nn(model, X_test_fifa, y_test_fifa, "FIFA")

Train on 8724 samples, validate on 2181 samples
Epoch 1/50
8724/8724 [==============================] - 4s 422us/step - loss: 1.4282 - categorical_accuracy: 0.4665 - val_loss: 1.0520 - val_categorical_accuracy: 0.4842
Epoch 2/50
8724/8724 [==============================] - 1s 154us/step - loss: 1.0605 - categorical_accuracy: 0.5170 - val_loss: 1.0588 - val_categorical_accuracy: 0.5589
Epoch 3/50
8724/8724 [==============================] - 1s 116us/step - loss: 1.0112 - categorical_accuracy: 0.5354 - val_loss: 0.9910 - val_categorical_accuracy: 0.5539
Epoch 4/50
8724/8724 [==============================] - 1s 110us/step - loss: 1.0359 - categorical_accuracy: 0.5291 - val_loss: 0.9653 - val_categorical_accuracy: 0.5328
Epoch 5/50
8724/8724 [==============================] - 1s 106us/step - loss: 0.9792 - categorical_accuracy: 0.5476 - val_loss: 0.9622 - val_categorical_accuracy: 0.5589
Epoch 6/50
8724/8724 [==============================] - 1s 98us/step - loss: 0.9694 - categorical_accu

8724/8724 [==============================] - 1s 112us/step - loss: 0.9238 - categorical_accuracy: 0.5726 - val_loss: 0.9548 - val_categorical_accuracy: 0.5630
Epoch 49/50
8724/8724 [==============================] - 1s 97us/step - loss: 0.9254 - categorical_accuracy: 0.5687 - val_loss: 0.9358 - val_categorical_accuracy: 0.5585
Epoch 50/50
8724/8724 [==============================] - 1s 101us/step - loss: 0.9296 - categorical_accuracy: 0.5699 - val_loss: 0.9413 - val_categorical_accuracy: 0.5585
Done
Neural Network Prediction Accuracy (FIFA): 53.125%
Train on 8724 samples, validate on 2181 samples
Epoch 1/50
8724/8724 [==============================] - 4s 456us/step - loss: 1.4748 - categorical_accuracy: 0.4731 - val_loss: 0.9907 - val_categorical_accuracy: 0.5580
Epoch 2/50
8724/8724 [==============================] - 1s 90us/step - loss: 1.0540 - categorical_accuracy: 0.5269 - val_loss: 0.9586 - val_categorical_accuracy: 0.5525
Epoch 3/50
8724/8724 [==============================] - 1

8724/8724 [==============================] - 1s 119us/step - loss: 0.9294 - categorical_accuracy: 0.5672 - val_loss: 0.9364 - val_categorical_accuracy: 0.5690
Epoch 46/50
8724/8724 [==============================] - 1s 124us/step - loss: 0.9273 - categorical_accuracy: 0.5672 - val_loss: 0.9387 - val_categorical_accuracy: 0.5621
Epoch 47/50
8724/8724 [==============================] - 1s 119us/step - loss: 0.9251 - categorical_accuracy: 0.5691 - val_loss: 0.9398 - val_categorical_accuracy: 0.5672
Epoch 48/50
8724/8724 [==============================] - 1s 148us/step - loss: 0.9252 - categorical_accuracy: 0.5661 - val_loss: 0.9422 - val_categorical_accuracy: 0.5676
Epoch 49/50
8724/8724 [==============================] - 2s 251us/step - loss: 0.9264 - categorical_accuracy: 0.5689 - val_loss: 0.9461 - val_categorical_accuracy: 0.5585
Epoch 50/50
8724/8724 [==============================] - 1s 103us/step - loss: 0.9236 - categorical_accuracy: 0.5713 - val_loss: 0.9318 - val_categorical_acc

8724/8724 [==============================] - 1s 97us/step - loss: 0.9333 - categorical_accuracy: 0.5629 - val_loss: 0.9319 - val_categorical_accuracy: 0.5663
Epoch 43/50
8724/8724 [==============================] - 1s 98us/step - loss: 0.9330 - categorical_accuracy: 0.5652 - val_loss: 0.9385 - val_categorical_accuracy: 0.5690
Epoch 44/50
8724/8724 [==============================] - 1s 115us/step - loss: 0.9279 - categorical_accuracy: 0.5665 - val_loss: 0.9333 - val_categorical_accuracy: 0.5630
Epoch 45/50
8724/8724 [==============================] - 2s 191us/step - loss: 0.9360 - categorical_accuracy: 0.5632 - val_loss: 0.9372 - val_categorical_accuracy: 0.5630
Epoch 46/50
8724/8724 [==============================] - 1s 122us/step - loss: 0.9283 - categorical_accuracy: 0.5690 - val_loss: 0.9374 - val_categorical_accuracy: 0.5594
Epoch 47/50
8724/8724 [==============================] - 1s 107us/step - loss: 0.9320 - categorical_accuracy: 0.5679 - val_loss: 0.9300 - val_categorical_accur

8724/8724 [==============================] - 1s 145us/step - loss: 0.9308 - categorical_accuracy: 0.5650 - val_loss: 0.9375 - val_categorical_accuracy: 0.5672
Epoch 40/50
8724/8724 [==============================] - 1s 168us/step - loss: 0.9296 - categorical_accuracy: 0.5676 - val_loss: 0.9326 - val_categorical_accuracy: 0.5695
Epoch 41/50
8724/8724 [==============================] - 1s 131us/step - loss: 0.9290 - categorical_accuracy: 0.5664 - val_loss: 0.9240 - val_categorical_accuracy: 0.5635
Epoch 42/50
8724/8724 [==============================] - 1s 88us/step - loss: 0.9296 - categorical_accuracy: 0.5642 - val_loss: 0.9526 - val_categorical_accuracy: 0.5571
Epoch 43/50
8724/8724 [==============================] - 1s 91us/step - loss: 0.9305 - categorical_accuracy: 0.5611 - val_loss: 0.9569 - val_categorical_accuracy: 0.5640
Epoch 44/50
8724/8724 [==============================] - 1s 117us/step - loss: 0.9332 - categorical_accuracy: 0.5614 - val_loss: 0.9475 - val_categorical_accur

...

8724/8724 [==============================] - 1s 110us/step - loss: 0.9300 - categorical_accuracy: 0.5660 - val_loss: 0.9304 - val_categorical_accuracy: 0.5658
Epoch 34/50
8724/8724 [==============================] - 1s 160us/step - loss: 0.9287 - categorical_accuracy: 0.5673 - val_loss: 0.9269 - val_categorical_accuracy: 0.5653
Epoch 35/50
8724/8724 [==============================] - 1s 144us/step - loss: 0.9337 - categorical_accuracy: 0.5627 - val_loss: 0.9429 - val_categorical_accuracy: 0.5667
Epoch 36/50
8724/8724 [==============================] - 1s 134us/step - loss: 0.9311 - categorical_accuracy: 0.5644 - val_loss: 0.9311 - val_categorical_accuracy: 0.5640
Epoch 37/50
8724/8724 [==============================] - 1s 123us/step - loss: 0.9268 - categorical_accuracy: 0.5681 - val_loss: 0.9318 - val_categorical_accuracy: 0.5708
Epoch 38/50
8724/8724 [==============================] - 1s 114us/step - loss: 0.9312 - categorical_accuracy: 0.5656 - val_loss: 0.9348 - val_categorical_acc

8724/8724 [==============================] - 1s 118us/step - loss: 0.9323 - categorical_accuracy: 0.5645 - val_loss: 0.9344 - val_categorical_accuracy: 0.5704
Epoch 31/50
8724/8724 [==============================] - 1s 103us/step - loss: 0.9334 - categorical_accuracy: 0.5630 - val_loss: 0.9324 - val_categorical_accuracy: 0.5649
Epoch 32/50
8724/8724 [==============================] - 1s 102us/step - loss: 0.9349 - categorical_accuracy: 0.5642 - val_loss: 0.9381 - val_categorical_accuracy: 0.5502
Epoch 33/50
8724/8724 [==============================] - 1s 134us/step - loss: 0.9327 - categorical_accuracy: 0.5649 - val_loss: 0.9315 - val_categorical_accuracy: 0.5690
Epoch 34/50
8724/8724 [==============================] - 1s 111us/step - loss: 0.9317 - categorical_accuracy: 0.5632 - val_loss: 0.9346 - val_categorical_accuracy: 0.5644
Epoch 35/50
8724/8724 [==============================] - 2s 216us/step - loss: 0.9278 - categorical_accuracy: 0.5687 - val_loss: 0.9307 - val_categorical_acc

8724/8724 [==============================] - 1s 84us/step - loss: 0.9345 - categorical_accuracy: 0.5679 - val_loss: 0.9392 - val_categorical_accuracy: 0.5644
Epoch 28/50
8724/8724 [==============================] - 1s 82us/step - loss: 0.9357 - categorical_accuracy: 0.5684 - val_loss: 0.9384 - val_categorical_accuracy: 0.5722
Epoch 29/50
8724/8724 [==============================] - 1s 86us/step - loss: 0.9347 - categorical_accuracy: 0.5636 - val_loss: 0.9388 - val_categorical_accuracy: 0.5649
Epoch 30/50
8724/8724 [==============================] - 1s 91us/step - loss: 0.9358 - categorical_accuracy: 0.5645 - val_loss: 0.9470 - val_categorical_accuracy: 0.5672
Epoch 31/50
8724/8724 [==============================] - 1s 86us/step - loss: 0.9338 - categorical_accuracy: 0.5634 - val_loss: 0.9464 - val_categorical_accuracy: 0.5621
Epoch 32/50
8724/8724 [==============================] - 1s 83us/step - loss: 0.9327 - categorical_accuracy: 0.5652 - val_loss: 0.9580 - val_categorical_accuracy:

Epoch 25/50
8724/8724 [==============================] - 1s 82us/step - loss: 0.9322 - categorical_accuracy: 0.5655 - val_loss: 0.9228 - val_categorical_accuracy: 0.5667
Epoch 26/50
8724/8724 [==============================] - 1s 83us/step - loss: 0.9343 - categorical_accuracy: 0.5652 - val_loss: 0.9294 - val_categorical_accuracy: 0.5676
Epoch 27/50
8724/8724 [==============================] - 1s 83us/step - loss: 0.9343 - categorical_accuracy: 0.5663 - val_loss: 0.9450 - val_categorical_accuracy: 0.5699
Epoch 28/50
8724/8724 [==============================] - 1s 83us/step - loss: 0.9330 - categorical_accuracy: 0.5669 - val_loss: 0.9400 - val_categorical_accuracy: 0.5598
Epoch 29/50
8724/8724 [==============================] - 1s 82us/step - loss: 0.9321 - categorical_accuracy: 0.5636 - val_loss: 0.9280 - val_categorical_accuracy: 0.5621
Epoch 30/50
8724/8724 [==============================] - 1s 83us/step - loss: 0.9320 - categorical_accuracy: 0.5624 - val_loss: 0.9430 - val_categoric

8724/8724 [==============================] - 1s 118us/step - loss: 0.9370 - categorical_accuracy: 0.5630 - val_loss: 0.9256 - val_categorical_accuracy: 0.5667
Epoch 23/50
8724/8724 [==============================] - 1s 113us/step - loss: 0.9345 - categorical_accuracy: 0.5648 - val_loss: 0.9304 - val_categorical_accuracy: 0.5580
Epoch 24/50
8724/8724 [==============================] - 1s 122us/step - loss: 0.9353 - categorical_accuracy: 0.5594 - val_loss: 0.9402 - val_categorical_accuracy: 0.5603
Epoch 25/50
8724/8724 [==============================] - 1s 116us/step - loss: 0.9366 - categorical_accuracy: 0.5619 - val_loss: 0.9340 - val_categorical_accuracy: 0.5589
Epoch 26/50
8724/8724 [==============================] - 1s 106us/step - loss: 0.9373 - categorical_accuracy: 0.5617 - val_loss: 0.9273 - val_categorical_accuracy: 0.5695
Epoch 27/50
8724/8724 [==============================] - 1s 110us/step - loss: 0.9308 - categorical_accuracy: 0.5625 - val_loss: 0.9288 - val_categorical_acc

In [73]:
sample_fifa

[53.125, 51.5625, 51.5625, 56.25, 56.25, 62.5, 53.125, 59.375, 50.0, 59.375]

In [79]:
print("FIFA Average Test Accuracy (10 Identical Neural Networks): {0}%".format(np.mean(sample_fifa)))

FIFA Average Test Accuracy (10 Identical Neural Networks): 55.3125%
